In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [2]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

checkpointer = MemorySaver()


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

tool = TavilySearchResults(max_results=2)
tools = [tool]
llm = ChatOpenAI()
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [ ]:
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")

In [5]:
graph = graph_builder.compile(
    checkpointer=checkpointer,
    interrupt_before=["tools"],
)

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="Hello, I am John")

graph.invoke({"messages": input_message}, config=config)

In [ ]:
config = {"configurable": {"thread_id": "100"}}
input_message = HumanMessage(content="Sorry, did I already introduce myself?")

graph.invoke({"messages": input_message}, config=config)

In [ ]:
config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="Sorry, did I already introduce myself?")

graph.invoke({"messages": input_message}, config=config)

In [ ]:
snapshot = graph.get_state(config)
snapshot.next

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="How is the weather in Los Angeles?")

graph.invoke({"messages": input_message}, config=config)

In [ ]:
snapshot = graph.get_state(config)
snapshot.next

In [ ]:
graph.invoke(None, config=config)

### Timetravel

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}
input_message = HumanMessage(content="How is the weather in Los Angeles?")

graph.invoke({"messages": input_message}, config=config)

In [ ]:
snapshot = graph.get_state(config)
existing_message = snapshot.values["messages"][-1]
existing_message.pretty_print()

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage

answer = "It is only 5°C warm today!"
new_messages = [
    ToolMessage(content=answer, tool_call_id=existing_message.tool_calls[0]["id"]),
    AIMessage(content=answer),
]
graph.update_state(
    config,
    {"messages": new_messages},
)

In [ ]:
print((graph.get_state(config).values["messages"]))

In [ ]:
config = {"configurable": {"thread_id": "2"}}
input_message = HumanMessage(content="How warm was it again?")

graph.invoke({"messages": input_message}, config=config)

### Replay

In [ ]:
all_checkpoints = []
for state in graph.get_state_history(config=config):
    all_checkpoints.append(state)
all_checkpoints

In [20]:
all_checkpoints = []
for state in graph.get_state_history(config=config):
    checkpoint_id = state.config.get('configurable', {}).get('checkpoint_id')
    if checkpoint_id:
        all_checkpoints.append(checkpoint_id)

# Get the first and latest checkpoint
first_checkpoint = all_checkpoints[0] if all_checkpoints else None
latest_checkpoint = all_checkpoints[-1] if all_checkpoints else None

In [ ]:
print("Id of first checkpoint: ", first_checkpoint)
print("Id of latest checkpoint: ", latest_checkpoint)

In [ ]:
config_latest = {'configurable': {'thread_id': '2', 'checkpoint_id': latest_checkpoint}}
graph.invoke({"messages": input_message}, config=config_latest)

In [ ]:
config_first = {'configurable': {'thread_id': '2', 'checkpoint_id': first_checkpoint}}
graph.invoke({"messages": input_message}, config=config_first)